In [1]:
from datasets import load_dataset

In [3]:
#dataset = load_dataset('json',data_files=['train.json','validation.json'])
dataset = load_dataset('json',data_files={'train':'../Datasets/train_gleif_ocdata_FULL.json','test':'../Datasets/test_gleif_ocdata_FULL.json'})

Using custom data configuration default-93e1d9f5dc119308


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:\Users\liuyw\.cache\huggingface\datasets\json\default-93e1d9f5dc119308\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset_input = load_dataset('json',data_files={'input':'../Datasets/input_gleif_ocdata_FULL.json'})

Using custom data configuration default-ef7507124f9d0361


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:\Users\liuyw\.cache\huggingface\datasets\json\default-ef7507124f9d0361\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 500
    })
})

In [6]:
dataset_input

DatasetDict({
    input: Dataset({
        features: ['idx', 'sentence1', 'sentence2'],
        num_rows: 97779
    })
})

In [24]:
dataset_input["input"][0:10]

{'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'sentence1': ['COL name VAL Schotterwerke Wager-Fischer GmbH & Co KG COL address VAL Adlerstr. 4, 89555 Steinheim COL number VAL Amtsgericht Steinheim HRA 540792 COL state VAL DE-BW',
  'COL name VAL Seidler Speditions-GmbH COL address VAL Wannenaeckerstr. 57, 74078 Heilbronn COL number VAL Amtsgericht Heilbronn HRB 103350 COL state VAL DE-BW',
  'COL name VAL Manfred Herwig Heizung + Sanitaer GmbH COL address VAL Industriestrasse 24, 37115 Duderstadt COL number VAL Amtsgericht Duderstadt HRB 102363 COL state VAL DE-NI',
  'COL name VAL Bueronord Medienvertretungen GmbH COL address VAL Gustav-Weihrauch-Weg 7, 22359 Hamburg COL number VAL Amtsgericht Hamburg HRB 37115 COL state VAL DE-HH',
  'COL name VAL Huelster GmbH & Co. Geschaeftseinrichtungs- und Immobilien-KG COL address VAL Am Brauhof 6, 53721 Siegburg COL number VAL Amtsgericht Siegburg HRA 1345 COL state VAL DE-NW',
  'COL name VAL Ceritech AG COL address VAL Karl-Rothe-Strasse 13, 0410

In [5]:
from transformers import AutoTokenizer

In [6]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
def tokenize_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],padding="max_length",truncation=True,max_length=128)

In [8]:
tokenized_datasets = dataset.map(tokenize_function,batched=True)
print(tokenized_datasets.column_names)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids'], 'test': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids']}


In [9]:
tokenized_datasets_input = dataset_input.map(tokenize_function,batched=True)
print(tokenized_datasets_input.column_names)

  0%|          | 0/98 [00:00<?, ?ba/s]

{'input': ['attention_mask', 'idx', 'input_ids', 'sentence1', 'sentence2', 'token_type_ids']}


In [10]:
from transformers import DataCollatorWithPadding

In [11]:
data_collator = DataCollatorWithPadding(tokenizer)

In [12]:
from transformers import BertForSequenceClassification

In [13]:
model = BertForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [15]:
from transformers import TrainingArguments

In [14]:
training_args = TrainingArguments("test-trainer")

In [16]:
from transformers import Trainer

trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json
Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json
Model weights saved in test-trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.24094536505805123, metrics={'train_runtime': 2843.7567, 'train_samples_per_second': 3.165, 'train_steps_per_second': 0.396, 'total_flos': 591999874560000.0, 'train_loss': 0.24094536505805123, 'epoch': 3.0})

In [66]:
predictions = trainer.predict(tokenized_datasets['test'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [67]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds = np.argmax(predictions.predictions,axis=1)

metric.compute(predictions=preds,references=predictions.label_ids)

{'f1': 0.8982725527831094}

In [68]:
metric_p = load_metric("precision")

preds = np.argmax(predictions.predictions,axis=1)

metric_p.compute(predictions=preds,references=predictions.label_ids)

{'precision': 0.8666666666666667}

In [69]:
metric_r = load_metric("recall")

preds = np.argmax(predictions.predictions,axis=1)

metric_r.compute(predictions=preds,references=predictions.label_ids)

{'recall': 0.9322709163346613}

In [70]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.7312623858451843,
 'eval_runtime': 44.2168,
 'eval_samples_per_second': 11.308,
 'eval_steps_per_second': 1.425,
 'epoch': 3.0}

<h1>Fine Tuning<h1>

In [17]:
training_args_with_epoch = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
from transformers import Trainer

trainer_eval_epoch = Trainer(
        model,
        training_args_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.17446133592393664, metrics={'train_runtime': 2872.7819, 'train_samples_per_second': 3.133, 'train_steps_per_second': 0.392, 'total_flos': 591999874560000.0, 'train_loss': 0.17446133592393664, 'epoch': 3.0})

In [19]:
predictions_eval_epoch = trainer_eval_epoch.predict(tokenized_datasets['test'])
print(predictions_eval_epoch.predictions.shape,predictions_eval_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'f1': 0.88671875}

In [21]:
metric_p = load_metric("precision")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'precision': 0.8697318007662835}

In [22]:
metric_r = load_metric("recall")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'recall': 0.9043824701195219}

In [23]:
trainer_eval_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.5882813930511475,
 'eval_runtime': 43.5842,
 'eval_samples_per_second': 11.472,
 'eval_steps_per_second': 1.445,
 'epoch': 3.0}

<h3>eval steps<h3>

In [24]:
training_args_with_steps = TrainingArguments("test_trainer", evaluation_strategy="steps")

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
from transformers import Trainer

trainer_eval_steps = Trainer(
        model,
        training_args_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.10763960350884332, metrics={'train_runtime': 2814.5156, 'train_samples_per_second': 3.198, 'train_steps_per_second': 0.4, 'total_flos': 591999874560000.0, 'train_loss': 0.10763960350884332, 'epoch': 3.0})

In [26]:
predictions_eval_steps = trainer_eval_steps.predict(tokenized_datasets['test'])
print(predictions_eval_steps.predictions.shape,predictions_eval_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [27]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'f1': 0.8965517241379309}

In [28]:
metric_p = load_metric("precision")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric_p.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'precision': 0.8634686346863468}

In [29]:
metric_r = load_metric("recall")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric_r.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'recall': 0.9322709163346613}

In [30]:
trainer_eval_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.6700529456138611,
 'eval_runtime': 39.3308,
 'eval_samples_per_second': 12.713,
 'eval_steps_per_second': 1.602,
 'epoch': 3.0}

<h3>eval epoch<h3>

In [31]:
training_args_with_epoch = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
from transformers import Trainer

trainer_eval_epoch = Trainer(
        model,
        training_args_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.07129681396484375, metrics={'train_runtime': 3070.1917, 'train_samples_per_second': 2.931, 'train_steps_per_second': 0.366, 'total_flos': 591999874560000.0, 'train_loss': 0.07129681396484375, 'epoch': 3.0})

In [33]:
predictions_eval_epoch = trainer_eval_epoch.predict(tokenized_datasets['test'])
print(predictions_eval_epoch.predictions.shape,predictions_eval_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [34]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'f1': 0.8978805394990366}

In [35]:
metric_p = load_metric("precision")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'precision': 0.8694029850746269}

In [36]:
metric_r = load_metric("recall")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'recall': 0.9282868525896414}

In [37]:
trainer_eval_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.6549674272537231,
 'eval_runtime': 49.7806,
 'eval_samples_per_second': 10.044,
 'eval_steps_per_second': 1.266,
 'epoch': 3.0}

<h3>Logging epoch<h3>

In [38]:
training_args_logging_with_epoch = TrainingArguments("test_trainer", logging_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
from transformers import Trainer

trainer_logging_epoch = Trainer(
        model,
        training_args_logging_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_logging_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.06516893683539497, metrics={'train_runtime': 2906.5216, 'train_samples_per_second': 3.096, 'train_steps_per_second': 0.387, 'total_flos': 591999874560000.0, 'train_loss': 0.06516893683539497, 'epoch': 3.0})

In [40]:
predictions_logging_epoch = trainer_logging_epoch.predict(tokenized_datasets['test'])
print(predictions_logging_epoch.predictions.shape,predictions_logging_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [41]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'f1': 0.9021113243761996}

In [42]:
metric_p = load_metric("precision")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'precision': 0.8703703703703703}

In [43]:
metric_r = load_metric("recall")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'recall': 0.9362549800796812}

In [44]:
trainer_logging_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.6494400501251221,
 'eval_runtime': 44.4735,
 'eval_samples_per_second': 11.243,
 'eval_steps_per_second': 1.417,
 'epoch': 3.0}

<h3>Logging steps<h3>

In [45]:
training_args_logging_with_steps = TrainingArguments("test_trainer", logging_strategy="steps")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [46]:
from transformers import Trainer

trainer_logging_steps = Trainer(
        model,
        training_args_logging_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_logging_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.06405754852294922, metrics={'train_runtime': 2859.8125, 'train_samples_per_second': 3.147, 'train_steps_per_second': 0.393, 'total_flos': 591999874560000.0, 'train_loss': 0.06405754852294922, 'epoch': 3.0})

In [47]:
predictions_logging_steps = trainer_logging_steps.predict(tokenized_datasets['test'])
print(predictions_logging_steps.predictions.shape,predictions_logging_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [48]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'f1': 0.8976377952755905}

In [49]:
metric_p = load_metric("precision")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric_p.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'precision': 0.8871595330739299}

In [50]:
metric_r = load_metric("recall")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric_r.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'recall': 0.9083665338645418}

In [51]:
trainer_logging_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.7335565090179443,
 'eval_runtime': 46.1313,
 'eval_samples_per_second': 10.839,
 'eval_steps_per_second': 1.366,
 'epoch': 3.0}

<h3>save epoch<h3>

In [16]:
training_args_save_with_epoch = TrainingArguments("test_trainer", save_strategy="epoch")

In [17]:
from transformers import Trainer

trainer_save_epoch = Trainer(
        model,
        training_args_save_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_save_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-375
Configuration saved in test_trainer\checkpoint-375\config.json
Model weights saved in test_trainer\checkpoint-375\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-375\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-375\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-750
Configuration saved in test_trainer\checkpoint-750\config.json
Model weights saved in test_trainer\checkpoint-750\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-750\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-750\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1125
Configuration saved in test_trainer\checkpoint-1125\config.json
Model weights saved in test_trainer\checkpoint-1125\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1125\tokenizer_config.json
Special tokens file saved in t

TrainOutput(global_step=1125, training_loss=0.24958714803059895, metrics={'train_runtime': 2513.4082, 'train_samples_per_second': 3.581, 'train_steps_per_second': 0.448, 'total_flos': 591999874560000.0, 'train_loss': 0.24958714803059895, 'epoch': 3.0})

In [54]:
predictions_save_epoch = trainer_save_epoch.predict(tokenized_datasets['test'])
print(predictions_save_epoch.predictions.shape,predictions_save_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [55]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'f1': 0.9111969111969114}

In [56]:
metric_p = load_metric("precision")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'precision': 0.8838951310861424}

In [57]:
metric_r = load_metric("recall")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'recall': 0.9402390438247012}

In [58]:
trainer_save_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.5655338168144226,
 'eval_runtime': 44.776,
 'eval_samples_per_second': 11.167,
 'eval_steps_per_second': 1.407,
 'epoch': 3.0}

<h3>save steps<h3>

In [59]:
training_args_save_with_steps = TrainingArguments("test_trainer", save_strategy="steps")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [60]:
from transformers import Trainer

trainer_save_steps = Trainer(
        model,
        training_args_save_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_save_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.0629859381781684, metrics={'train_runtime': 3071.6009, 'train_samples_per_second': 2.93, 'train_steps_per_second': 0.366, 'total_flos': 591999874560000.0, 'train_loss': 0.0629859381781684, 'epoch': 3.0})

In [61]:
predictions_save_steps = trainer_save_steps.predict(tokenized_datasets['test'])
print(predictions_save_steps.predictions.shape,predictions_save_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [62]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'f1': 0.8982725527831094}

In [63]:
metric_p = load_metric("precision")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric_p.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'precision': 0.8666666666666667}

In [64]:
metric_r = load_metric("recall")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric_r.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'recall': 0.9322709163346613}

In [65]:
trainer_save_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.7312623858451843,
 'eval_runtime': 45.5528,
 'eval_samples_per_second': 10.976,
 'eval_steps_per_second': 1.383,
 'epoch': 3.0}

In [19]:
predictions = trainer.predict(tokenized_datasets['test'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds = np.argmax(predictions.predictions,axis=1)

metric.compute(predictions=preds,references=predictions.label_ids)

{'f1': 0.9029850746268656}

In [19]:
predictions_input = trainer_save_epoch.predict(tokenized_datasets_input['input'])
#print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 97779
  Batch size = 8


In [21]:
import numpy as np
preds = np.argmax(predictions_input.predictions,axis=1)
len(preds)

97779

In [22]:
with open('../Datasets/final_Bert_gleif_ocdata_FULL.txt', 'w') as f:
    for item in preds:
        f.write("%s\n" % item)